In [1]:
import tensorflow as tf
from tensorflow import keras

2022-08-09 13:52:00.611839: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-09 13:52:00.611875: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [ ]:
epochs = 1000
num_batches = 250
batch_size = 2

#Fixed parameters
def lr_schedule(epoch,lr):
    lr *=1-(epoch/epochs)**0.9
    return lr

lr = tf.keras.callbacks.LearningRateScheduler(lr_schedule)

def DiceLoss(targets, inputs, smooth=1e-6):
    
    #flatten label and prediction tensors
    inputs = keras.backend.flatten(inputs)
    targets = keras.backend.flatten(targets)
    
    intersection = keras.backend.sum(keras.backend.dot(targets, inputs))
    dice = (2*intersection + smooth) / (keras.backend.sum(targets) + keras.backend.sum(inputs) + smooth)
    return 1 - dice

loss1 = keras.losses.CategoricalCrossentropy()
loss2 = DiceLoss()
loss = loss1 + loss2

Optimizer = keras.optimizer.SGD(lr,momentum=0.99,nesterov=True, name='SGD')



In [ ]:
def NN_UNet():
    inputs = keras.Input()

    #Downsampeling
    x = keras.layers.Conv2D(32,3)(inputs)
    x = keras.layers.Normalization()(x)
    x = keras.layers.LeakyReLU(0.01)(x)
    l1 = x

    x = keras.layers.Conv2D(64,3)(x)
    x = keras.layers.Normalization()(x)
    x = keras.layers.LeakyReLU(0.01)(x)
    l2 = x

    x = keras.layers.Conv2D(128,3)(x)
    x = keras.layers.Normalization()(x)
    x = keras.layers.LeakyReLU(0.01)(x)
    l3 = x

    x = keras.layers.Conv2D(256,3)(x)
    x = keras.layers.Normalization()(x)
    x = keras.layers.LeakyReLU(0.01)(x)
    l4 = x

    x = keras.layers.Conv2D(512,3)(x)
    x = keras.layers.Normalization()(x)
    x = keras.layers.LeakyReLU(0.01)(x)
    l5 = x
    #Upsampleing

    x = keras.layers.Conv2DTranspose(512,3)(x)
    x = keras.layers.Normalization()(x)
    x = keras.layers.LeakyReLU(0.01)(x)
    l1 = x
